<a href="https://colab.research.google.com/github/MontiMic/data/blob/main/Concurrency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programmazione Concorrente

In [ ]:
import threading
import time

Cominciamo definendo una funzione che stampa 5 volte la stringa "Hello, World!"

In [ ]:
def hello_world():
  for i in range(5):
    print("Hello, World!")

In questo momento la funzione esiste, ma nessuno la sta usando. Creiamo un thread il cui scopo (o target) sarà di richiamare la funzione *hello_world()* e poi morire.

In [ ]:
mythread = threading.Thread(target=hello_world)   #Creo il thread e gli assegno del lavoro
mythread.start()  #Lancio il thread
mythread.join()   #Aspetto che abbia finito

Hello, World!
Hello, World!
Hello, World!
Hello, World!
Hello, World!


##Ping pong game
Ora pensiamo di voler creare un gioco dove due utenti devono rimbalzarsi la palla avanti e indietro.

Per farlo creiamo due thread che lanceremo parallelamente. Purtroppo non abbbiamo nessuna garanzia che sia rispettato l'ordine che desideriamo, e di fatto, se i processi hanno durate diverse incorreremo in questo errore.

In [ ]:
def play_ping():
    for i in range(5):
        print("Ping")
        time.sleep(1)   #Il primo thread ha una sleep di un secondo

def play_pong():
    for i in range(5):
        print("Pong")
        time.sleep(3)   #Il secondo ha una sleep di 3 secondi

In [ ]:
ping_thread = threading.Thread(target=play_ping)
pong_thread = threading.Thread(target=play_pong)

ping_thread.start()
pong_thread.start()

ping_thread.join()
pong_thread.join()

Ping
Pong
Ping
Ping
Ping
Pong
Ping
Pong
Pong
Pong


Soluzione: definiamo la mutex ed un flag:

In [ ]:
class PingPongGame:
    def __init__(self):
        self.mutex = threading.Lock()
        self.ping_turn = True

    def play_ping(self):
        for i in range(5):
            with self.mutex:
                while not self.ping_turn:
                    self.mutex.release()
                    time.sleep(0.1)  # Sleep to avoid busy waiting
                    self.mutex.acquire()

                print("Ping")
                self.ping_turn = False

            time.sleep(1)

    def play_pong(self):
        for i in range(5):
            with self.mutex:
                while self.ping_turn:
                    self.mutex.release()
                    time.sleep(0.1)  # Sleep to avoid busy waiting
                    self.mutex.acquire()

                print("Pong")
                self.ping_turn = True

            time.sleep(3)

In [ ]:
game = PingPongGame()

ping_thread = threading.Thread(target=game.play_ping)
pong_thread = threading.Thread(target=game.play_pong)

ping_thread.start()
pong_thread.start()

ping_thread.join()
pong_thread.join()

Ping
Pong
Ping
Pong
Ping
Pong
Ping
Pong
Ping
Pong


Perchè servono sia la mutex sia il flag? Non ne basta uno solo dei due?

Esercizio:
Sulla traccia del gioco ping pong aiutiamo Fra Martino a suonare le campane